In [1]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests

API_KEY = {'X-API-key': 'QA8I3P51'}
s = requests.Session()
s.headers.update(API_KEY)
resp = s.get('http://localhost:9999/v1/case')
resp

<Response [200]>

In [3]:
# basic api calls/functions

def send_order(sym, side, price, size):
    resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': sym, 'type': 'LIMIT', 'action': side, 'quantity': size, 'price': price})
    if resp.ok:
        print('SENT:', side, sym, size, '@', price)
        # print('Order ID:', resp.json()['order_id'])
    else:
        print('failed to send order', side, sym, size, '@', price, ':', resp.text)

def get_book(ticker):
    resp = s.get(
        'http://localhost:9999/v1/securities/book',
        params={'ticker': ticker}
    )
    resp.raise_for_status()
    return resp.json()

def best_bid_ask(book):
    bids = book.get('bids', [])
    asks = book.get('asks', [])

    best_bid = bids[0]['price'] if bids else None
    best_bid_qty = bids[0]['quantity'] if bids else 0

    best_ask = asks[0]['price'] if asks else None
    best_ask_qty = asks[0]['quantity'] if asks else 0

    return best_bid, best_ask, best_bid_qty, best_ask_qty

def get_positions():
    resp = s.get('http://localhost:9999/v1/securities')
    resp.raise_for_status()

    positions = {}
    for sec in resp.json():
        positions[sec['ticker']] = sec['position']
    return positions

def get_open_orders():
    resp = s.get('http://localhost:9999/v1/orders')
    resp.raise_for_status()
    return resp.json()

def get_case():
    resp = s.get('http://localhost:9999/v1/case')
    resp.raise_for_status()
    return resp.json()

def we_own_best_bid_ask(symbol, best_bid, best_ask):
    orders = get_open_orders()

    own_best_bid = False
    own_best_ask = False

    for o in orders:
        if o.get('status') != 'OPEN':
            continue
        if o.get('ticker') != symbol:
            continue

        if o.get('action') == 'BUY' and best_bid is not None:
            if o.get('price') == best_bid:
                own_best_bid = True

        if o.get('action') == 'SELL' and best_ask is not None:
            if o.get('price') == best_ask:
                own_best_ask = True

        if own_best_bid and own_best_ask:
            break

    return own_best_bid, own_best_ask


In [4]:
# arbitrage with fruit vs apple and orange sum

def frt_v_ao(buffer=5):
    A = get_book('APPL')
    O = get_book('ORNG')
    F = get_book('FRUIT')

    A_bid, A_ask, A_bid_q, A_ask_q = best_bid_ask(A)
    O_bid, O_ask, O_bid_q, O_ask_q = best_bid_ask(O)
    F_bid, F_ask, F_bid_q, F_ask_q = best_bid_ask(F)

    if None in [A_bid, A_ask, O_bid, O_ask, F_bid, F_ask]:
        return

    # buy if fruit underpriced
    if F_ask < A_bid + O_bid - buffer:
        size = min(F_ask_q, A_bid_q, O_bid_q, 5)
        if size > 0:
            print("buy frt")
            send_order('FRUIT', 'BUY',  F_ask, size)
            send_order('APPL',  'SELL', A_bid, size)
            send_order('ORNG',  'SELL', O_bid, size)

    # sell if fruit overpriced
    elif F_bid > A_ask + O_ask + buffer:
        size = min(F_bid_q, A_ask_q, O_ask_q, 5)
        if size > 0:
            print("sell frt")
            send_order('FRUIT', 'SELL', F_bid, size)
            send_order('APPL',  'BUY',  A_ask, size)
            send_order('ORNG',  'BUY',  O_ask, size)


In [5]:
# approx fruit fair value

N_TEAMS = 10

TEAMS = range(N_TEAMS)

team_belief = {
    team_id: {"L": 0.0, "U": 2000.0}
    for team_id in range(10)
    if team_id != 9
}

def update_team_belief(team_id, symbol, side, price):
    if symbol != 'FRUIT':
        return

    b = team_belief[team_id]

    if side == 'BUY':
        b["L"] = max(b["L"], price)

    elif side == 'SELL':
        b["U"] = min(b["U"], price)

    if b["L"] > b["U"]:
        mid = 0.5 * (b["L"] + b["U"])
        b["L"] = mid
        b["U"] = mid

def market_implied_fv():
    num, den = 0.0, 0.0

    for b in team_belief.values():
        width = b["U"] - b["L"]
        if width <= 0:
            continue

        center = 0.5 * (b["L"] + b["U"])
        weight = 1.0 / width

        num += center * weight
        den += weight

    return num / den if den > 0 else None

# Priors
mu_appl = 50.0
mu_orng = 50.0

# Confidence weights
conf_appl = 1.0
conf_orng = 1.0

# Your private info
my_appl = None
my_orng = None

def expected_appl():
    if my_appl is None:
        return N_TEAMS * mu_appl
    return my_appl + (N_TEAMS - 1) * mu_appl

def expected_orng():
    if my_orng is None:
        return N_TEAMS * mu_orng
    return my_orng + (N_TEAMS - 1) * mu_orng

def fundamental_fruit_fv():
    return expected_appl() + expected_orng()

def update_from_private(appl=None, orng=None):
    global my_appl, my_orng
    if appl is not None:
        my_appl = appl
    if orng is not None:
        my_orng = orng

def update_from_aggregate(sum_appl_5=None, sum_orng_5=None):
    global mu_appl, mu_orng, conf_appl, conf_orng

    if sum_appl_5 is not None:
        sample_mean = sum_appl_5 / 5
        weight = 4.0
        mu_appl = (conf_appl * mu_appl + weight * sample_mean) / (conf_appl + weight)
        conf_appl += weight

    if sum_orng_5 is not None:
        sample_mean = sum_orng_5 / 5
        weight = 4.0
        mu_orng = (conf_orng * mu_orng + weight * sample_mean) / (conf_orng + weight)
        conf_orng += weight

def blended_fruit_fv(alpha=0.7):
    fund_fv = fundamental_fruit_fv()
    market_fv = market_implied_fv()

    if market_fv is None:
        return fund_fv

    return alpha * fund_fv + (1 - alpha) * market_fv


In [6]:
def tighten_spread_if_not_us(symbol, size=5, min_spread=2):
    book = get_book(symbol)
    bid, ask, _, _ = best_bid_ask(book)

    if bid is None or ask is None:
        return

    spread = ask - bid
    if spread < min_spread:
        return

    own_bid, own_ask = we_own_best_bid_ask(symbol, bid, ask)

    if not own_bid:
        buy_price = bid + 5
        if buy_price < ask:
            send_order(symbol, 'BUY', buy_price, size)

    if not own_ask:
        sell_price = ask - 5
        if sell_price > bid:
            send_order(symbol, 'SELL', sell_price, size)

def decay_team_beliefs(decay=0.995):
    for b in team_belief.values():
        L = b["L"]
        U = b["U"]

        center = 0.5 * (L + U)
        half_width = 0.5 * (U - L)

        # Widen interval by 1/decay
        new_half_width = half_width / decay

        b["L"] = max(0.0, center - new_half_width)
        b["U"] = min(2000.0, center + new_half_width)


In [ ]:
import time

prev_best = {
    'FRUIT': {'bid': None, 'ask': None}
}

last_decay_time = time.time()
last_mm_time = 0.0
seeded = False

MARKET = 0

while True:
    try:
        print(team_belief)
        print("Fund FV:", fundamental_fruit_fv())
        print("Market FV:", market_implied_fv())
        print()
        print()
        print()
        case = get_case()
        if case['status'].lower() != 'active':
            time.sleep(0.5)
            continue

        tick = case['tick']
        now = time.time()

        if tick < 360:
            arb_buffer = 8
            fv_buffer  = None
            alpha = 0.8
            do_mm_appl = True
            do_mm_orng = True
        elif tick < 720:
            arb_buffer = 6
            fv_buffer  = 12
            alpha = 0.6
            do_mm_appl = True
            do_mm_orng = True
        else:
            arb_buffer = 4
            fv_buffer  = 6
            alpha = 0.4
            do_mm_appl = True
            do_mm_orng = True

        F_book = get_book('FRUIT')
        F_bid, F_ask, _, _ = best_bid_ask(F_book)

        frt_v_ao(buffer=arb_buffer)

        if prev_best['FRUIT']['bid'] is not None and prev_best['FRUIT']['ask'] is not None:
            prev_bid = prev_best['FRUIT']['bid']
            prev_ask = prev_best['FRUIT']['ask']

            own_bid, own_ask = we_own_best_bid_ask('FRUIT', prev_bid, prev_ask)

            if (not own_ask) and (F_ask is not None) and (prev_ask is not None):
                if F_ask > prev_ask:
                    update_team_belief(
                        team_id=MARKET,
                        symbol='FRUIT',
                        side='BUY',
                        price=prev_ask
                    )

            if (not own_bid) and (F_bid is not None) and (prev_bid is not None):
                if F_bid < prev_bid:
                    update_team_belief(
                        team_id=MARKET,
                        symbol='FRUIT',
                        side='SELL',
                        price=prev_bid
                    )
                    
        prev_best['FRUIT']['bid'] = F_bid
        prev_best['FRUIT']['ask'] = F_ask

        if now - last_mm_time > 0.5:
            tighten_spread_if_not_us('FRUIT', size=3)

            if do_mm_appl:
                tighten_spread_if_not_us('APPL', size=3)

            if do_mm_orng:
                tighten_spread_if_not_us('ORNG', size=3)

            last_mm_time = now

        if fv_buffer is not None and F_bid is not None and F_ask is not None:
            fv = blended_fruit_fv(alpha)

            if F_ask < fv - fv_buffer:
                send_order('FRUIT', 'BUY', F_ask, 5)

            elif F_bid > fv + fv_buffer:
                send_order('FRUIT', 'SELL', F_bid, 5)
                
        if now - last_decay_time > 1.0:
            decay_team_beliefs()
            last_decay_time = now

        time.sleep(0.1)

    except KeyboardInterrupt:
        print("Stopped by user")
        break

    except Exception as e:
        print("Error:", e)
        time.sleep(0.2)

{0: {'L': 0.0, 'U': 2000.0}, 1: {'L': 0.0, 'U': 2000.0}, 2: {'L': 0.0, 'U': 2000.0}, 3: {'L': 0.0, 'U': 2000.0}, 4: {'L': 0.0, 'U': 2000.0}, 5: {'L': 0.0, 'U': 2000.0}, 6: {'L': 0.0, 'U': 2000.0}, 7: {'L': 0.0, 'U': 2000.0}, 8: {'L': 0.0, 'U': 2000.0}}
Fund FV: 1000.0
Market FV: 999.9999999999999



{0: {'L': 0.0, 'U': 2000.0}, 1: {'L': 0.0, 'U': 2000.0}, 2: {'L': 0.0, 'U': 2000.0}, 3: {'L': 0.0, 'U': 2000.0}, 4: {'L': 0.0, 'U': 2000.0}, 5: {'L': 0.0, 'U': 2000.0}, 6: {'L': 0.0, 'U': 2000.0}, 7: {'L': 0.0, 'U': 2000.0}, 8: {'L': 0.0, 'U': 2000.0}}
Fund FV: 1000.0
Market FV: 999.9999999999999



{0: {'L': 0.0, 'U': 2000.0}, 1: {'L': 0.0, 'U': 2000.0}, 2: {'L': 0.0, 'U': 2000.0}, 3: {'L': 0.0, 'U': 2000.0}, 4: {'L': 0.0, 'U': 2000.0}, 5: {'L': 0.0, 'U': 2000.0}, 6: {'L': 0.0, 'U': 2000.0}, 7: {'L': 0.0, 'U': 2000.0}, 8: {'L': 0.0, 'U': 2000.0}}
Fund FV: 1000.0
Market FV: 999.9999999999999



{0: {'L': 0.0, 'U': 2000.0}, 1: {'L': 0.0, 'U': 2000.0}, 2: {'L': 0.0, 'U': 2000.0}, 3: {'L': 0.0